    import required libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate

import seaborn as sns

    functions to preprocess a feature of data

In [ ]:
 def encode_coapplicant(x):
    if x == '1':
        return 1
    else:
        return 0

    Selecting most relevent and important features, after performing data preprocessing 
    and then using feature selection, these feature are most important to get high accuracy

In [ ]:
COLUMNS = ['Age', 'Loan Amount Request (USD)', 'Credit Score',
           'Property Age', 'Co-Applicant', 'Loan Sanction Amount (USD)']

    reading and preprocessing the training data

In [ ]:
training_data = pd.read_csv('../input/predict-loan-amount-data/train.csv', usecols=COLUMNS)

indexes = training_data[training_data['Loan Sanction Amount (USD)'].isna()].index
training_data.drop(index=indexes, axis=0, inplace=True)

training_data['Loan Sanction Amount (USD)'].fillna(0, inplace=True)

indexes = training_data[training_data['Loan Sanction Amount (USD)'] < 0].index
training_data.loc[indexes,'Loan Sanction Amount (USD)'] = 0


training_data.loc[(training_data['Co-Applicant'] == -999),'Co-Applicant'] = 0

training_data['Property Age'].fillna(training_data['Property Age'].mean().round(2), inplace=True)
training_data['Credit Score'].fillna(0, inplace=True)

print(training_data.shape)
training_data.head(3)

    Training data visuablisation

In [ ]:
sns.heatmap(training_data.corr(),annot=True)

In [ ]:
sns.pairplot(training_data)

    Extracting target labels/values to other variable, and then splitting the data into train and test set,
    on which model will be trained and evaluated

In [ ]:
target_loan = training_data['Loan Sanction Amount (USD)']
training_data.drop(columns=['Loan Sanction Amount (USD)'], axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(training_data, target_loan, test_size=0.2, random_state=21)

    Training the GradientBoostingRegressor, and using the test set as the validation set to check the performance of the model

In [ ]:
gbr = GradientBoostingRegressor(warm_start=True, criterion='mse',max_depth=5, n_estimators=150, n_iter_no_change=10 ,verbose=0)

gbr.fit(x_train, y_train)
predictions = gbr.predict(x_test)
print('r2 : ',max(0,100*r2_score(y_test, predictions)))
print('mse : ',mean_squared_error(y_test, predictions))
print("score : ",gbr.score(x_train, y_train))

    Performing cross-validation to see how the model performs on whole training data

In [ ]:
cross_validate(gbr, training_data, target_loan, cv=6, scoring='r2', verbose=0)

    reading and preprocessing the test data


In [ ]:
test_data = pd.read_csv('../input/predict-loan-amount-data/test.csv')
CUST_ID = test_data['Customer ID']
COLUMNS.remove('Loan Sanction Amount (USD)')
test_data = pd.read_csv('../input/predict-loan-amount-data/test.csv',usecols=COLUMNS)

test_data.loc[(test_data['Co-Applicant'] == '?'),'Co-Applicant'] = '0'
test_data['Co-Applicant'] = test_data['Co-Applicant'].astype(int)

test_data['Property Age'].fillna(training_data['Property Age'].mean().round(2), inplace=True)
test_data['Credit Score'].fillna(0, inplace=True)

print(test_data.shape)
test_data.head(3)

    generating predictions on the test data, and writing those to a submission file

In [ ]:
test_pred = gbr.predict(test_data)

In [ ]:
submission = pd.DataFrame({'Customer ID' : CUST_ID, 'Loan Sanction Amount (USD)':test_pred})
submission.to_csv('submission.csv',index=False)